In [9]:
from keras.optimizers import Adam
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.models import Model,Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Input, concatenate, Conv2DTranspose, Conv2D, MaxPooling2D
import numpy as np
import random
from keras import backend as K
import cv2
import os
import pickle
np.random.seed(2017)

In [11]:
#Checking if pickled data of images exists
print("Checking if pickled data of images exists")
data = []
label = []
if os.path.exists('pickled_data/segmentation.pkl'):
    print("Pickled data of images exist")
    dataset = pickle.load(open( 'pickled_data/segmentation.pkl', 'rb'))
    data = dataset['data']
    label = dataset['label']
else:
    print("Pickled data doesn't exist, loading data")
    if not os.path.exists('pickled_data'):
        os.makedirs('pickled_data')
    directory = os.fsencode("melanoma")
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        path = str('melanoma/' + filename)
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        data.append(image)
        
        path = str('gt/' + filename.split('.')[0] + '_segmentation.png')
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        label.append(image)

    directory = os.fsencode("others")
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        path = str('others/' + filename)
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        data.append(image)
        
        path = str('gt/' + filename.split('.')[0] + '_segmentation.png')
        image = cv2.imread(path)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        label.append(image)
    dataset = {}
    dataset['data'] = data
    dataset['label'] = label
    pickle.dump(dataset, open('pickled_data/segmentation.pkl', 'wb'))
print("Data is loaded")

Checking if pickled data of images exists
Pickled data of images exist
Data is loaded


In [12]:
#Converting data and label in NP arrays and Normalising data 
data = np.array(data, dtype="float") / 255.0
label = np.array(label, dtype="float") / 255.0

#Splitting into train and test dataset
(x_train, x_test, y_train, y_test) = train_test_split(data,label, test_size=0.20, random_state=2018)


In [ ]:
batch_size = 32
num_classes = 2

epochs = 10

input_shape = (64,64,3)

inputs = Input(input_shape)

conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(3, (1, 1), activation='softmax')(conv9)

model = Model(inputs=[inputs], outputs=[conv10])

model.compile(optimizer=Adam(lr=1e-5), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Score:', score)

Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 454s - loss: 0.6395 - acc: 0.9541 - val_loss: 0.6396 - val_acc: 0.8099
Epoch 2/10
1600/1600 [==============================] - 432s - loss: 0.6393 - acc: 0.4277 - val_loss: 0.6395 - val_acc: 0.2451
Epoch 3/10
1600/1600 [==============================] - 458s - loss: 0.6393 - acc: 0.2667 - val_loss: 0.6395 - val_acc: 0.2845
Epoch 4/10
1600/1600 [==============================] - 438s - loss: 0.6393 - acc: 0.3114 - val_loss: 0.6395 - val_acc: 0.3561
Epoch 5/10
1600/1600 [==============================] - 591s - loss: 0.6393 - acc: 0.4005 - val_loss: 0.6395 - val_acc: 0.4160
Epoch 6/10
1600/1600 [==============================] - 538s - loss: 0.6393 - acc: 0.4349 - val_loss: 0.6395 - val_acc: 0.4337
Epoch 7/10
1600/1600 [==============================] - 463s - loss: 0.6393 - acc: 0.4486 - val_loss: 0.6395 - val_acc: 0.4543
Epoch 8/10
1600/1600 [==============================] - 433s - l